In [1]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
import re

In [2]:
#Get all sentences
dataset_sentence = [" ".join(line.split("|"))[:-1] for line in open('SOStr.txt')]
sentences = [sent for sent in dataset_sentence]

#Get from phrase to phrase id
dataset_phrase_id = pd.read_csv('dictionary.txt', sep='|', names = ["phrase", "id"])
dict_phrase_id = dict([[row['phrase'], row['id']] for _,row in dataset_phrase_id.iterrows()])

#Get from phrase id to sentiment score
sentiment_phrases = pd.read_csv('sentiment_labels.txt', sep='|')
dict_id_sentiment = dict([[int(row['phrase ids']), row['sentiment values']] for _,row in sentiment_phrases.iterrows()])
sentence_sentiment = [[sentence, dict_id_sentiment[dict_phrase_id[sentence]]] for sentence in sentences]



In [3]:
dataset_split = pd.read_csv('datasetSplit.txt', sep=',')
splitset_label = list(dataset_split['splitset_label'])

In [4]:
src_train = [sentence_sentiment[i][0] for i in range(len(splitset_label)) if splitset_label[i] == 1]
tgt_train = [sentence_sentiment[i][1] for i in range(len(splitset_label)) if splitset_label[i] == 1]

src_test = [sentence_sentiment[i][0] for i in range(len(splitset_label)) if splitset_label[i] == 2]
tgt_test = [sentence_sentiment[i][1] for i in range(len(splitset_label)) if splitset_label[i] == 2]

src_val = [sentence_sentiment[i][0] for i in range(len(splitset_label)) if splitset_label[i] == 3]
tgt_val = [sentence_sentiment[i][1] for i in range(len(splitset_label)) if splitset_label[i] == 3]


In [5]:
#Binarize
tgt_sets = [tgt_train, tgt_test, tgt_val]

for l in tgt_sets:
    for i in range(len(l)):
        l[i] = int(l[i]*5)+1

In [6]:
def remove_figures(sentence):
    replaced = re.sub('[0-9]([0-9]|\.|,|;| \ [0-9])*', 'NUMTOKEN', sentence)
    return replaced

def clean_text(text):
    text = text.lower()
    text = " ".join(word_tokenize(text))
    text = remove_figures(text)
    return text

#Clean texts
src_sets = [src_train, src_test, src_val]

for l in src_sets:
    for i in range(len(l)):
        l[i] = clean_text(l[i])

In [7]:
file_contents = [src_train, tgt_train, src_test, tgt_test, src_val, tgt_val]
file_names = ['src_train', 'tgt_train', 'src_test', 'tgt_test', 'src_val', 'tgt_val']

for i in range(len(file_contents)):
    with open(file_names[i]+".txt", 'w') as f:
        for line in file_contents[i]:
            f.write(str(line)+"\n")



### CHARACTER LEVEL

In [8]:
char_src_train = []
char_src_test = [] 
char_src_val = []

src_sets = [src_train, src_test, src_val]
char_src_sets = [char_src_train, char_src_test, char_src_val]

for i in range(len(src_sets)):
    for sent in src_sets[i]:
        char_sent = " ".join(sent).replace("N U M T O K E N", "NUMTOKEN").replace("   ", " SP ")
        char_src_sets[i].append(char_sent)
    

In [9]:
file_contents = [char_src_train, char_src_test, char_src_val]
file_names = ['char_src_train', 'char_src_test', 'char_src_val']

import os
if not os.path.exists('char'):
    os.makedirs('char')

for i in range(len(file_contents)):
    with open('char/'+file_names[i]+".txt", 'w') as f:
        for line in file_contents[i]:
            f.write(str(line)+"\n")